Fonte: https://github.com/jackfrost1411/next_word_suggestor


Para executar o código a seguir, é preciso entrar no link acima, baixar o arquivo doc3.py e carrega-lo neste projeto. 

In [5]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re # para mexer com regexp
from tensorflow.keras.utils import to_categorical
from doc3 import training_doc3

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
print(training_doc3)


how many people live in atlanta georgia	Atlanta ( , stressed , locally ) is the capital of and the most populous city in the U.S. state of Georgia , with an estimated 2011 population of 432,427 .	1
how many people live in atlanta georgia	Atlanta is the cultural and economic center of the Atlanta metropolitan area , home to 5,457,831 people and the ninth largest metropolitan area in the United States .	1
how many people live in atlanta georgia	Atlanta is the county seat of Fulton County , and a small portion of the city extends eastward into DeKalb County .	0
how many people live in atlanta georgia	Atlanta was established in 1837 at the intersection of two railroad lines , and the city rose from the ashes of the Civil War to become a national center of commerce .	0
how many people live in atlanta georgia	In the decades following the Civil Rights Movement , during which the city earned a reputation as `` too busy to hate '' for the progressive views of its citizens and leaders , Atlanta

In [8]:
cleaned = re.sub(r'\W+', ' ', training_doc3).lower()
tokens = word_tokenize(cleaned)

train_len = 4 # vamos usar 4 aqui e é esperado que a gente use 3 palavras na hora de fazer query
text_sequences = []
for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences), train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]


In [9]:
k = np.random.choice(range(len(n_sequences)))
for i in sequences[k]:
  print(tokens[i])

in
live
state
the


In [10]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size) # one hot encoding
seq_len = train_inputs.shape[1]

print(train_inputs.shape)

(1392, 3)


In [11]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 3)              1365      
                                                                 
 lstm (LSTM)                 (None, 3, 50)             10800     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 455)               23205     
                                                                 
Total params: 58,120
Trainable params: 58,120
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
model.fit(train_inputs,train_targets,epochs=500,verbose=1)
model.save("mymodel.h5")

Epoch 1/500
44/44 [==============================] - 5s 8ms/step - loss: 6.0680 - accuracy: 0.0740
Epoch 2/500
44/44 [==============================] - 0s 8ms/step - loss: 5.3756 - accuracy: 0.0797
Epoch 3/500
44/44 [==============================] - 0s 8ms/step - loss: 5.0653 - accuracy: 0.0797
Epoch 4/500
44/44 [==============================] - 0s 9ms/step - loss: 4.9598 - accuracy: 0.0797
Epoch 5/500
44/44 [==============================] - 0s 8ms/step - loss: 4.8805 - accuracy: 0.0783
Epoch 6/500
44/44 [==============================] - 0s 8ms/step - loss: 4.8196 - accuracy: 0.0812
Epoch 7/500
44/44 [==============================] - 0s 8ms/step - loss: 4.7789 - accuracy: 0.0898
Epoch 8/500
44/44 [==============================] - 0s 8ms/step - loss: 4.7350 - accuracy: 0.0912
Epoch 9/500
44/44 [==============================] - 0s 8ms/step - loss: 4.6777 - accuracy: 0.1013
Epoch 10/500
44/44 [==============================] - 0s 9ms/step - loss: 4.5881 - accuracy: 0.1092
Epoch 11/

In [13]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

bird
[] [[0 0 0]]
Next word suggestion: 1996
Next word suggestion: destroyed
Next word suggestion: presence


# Usando um texto do Kafka:

In [19]:
import requests

texto = requests.get('https://www.gutenberg.org/cache/epub/5200/pg5200.txt').text
print(texto)

﻿The Project Gutenberg EBook of Metamorphosis, by Franz Kafka
Translated by David Wyllie.

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org

** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **
**     Please follow the copyright guidelines in this file.     **


Title: Metamorphosis

Author: Franz Kafka

Translator: David Wyllie

Release Date: August 16, 2005 [EBook #5200]
First posted: May 13, 2002
Last updated: May 20, 2012

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK METAMORPHOSIS ***




Copyright (C) 2002 David Wyllie.





  Metamorphosis
  Franz Kafka

Translated by David Wyllie



I


One morning, when Gregor Samsa woke from troubled dreams, he found
himself transformed in his bed into a horrible vermin.  He lay on
his armour-like back, and if he lif

In [20]:
cleaned = re.sub(r'\W+', ' ', texto).lower()
tokens = word_tokenize(cleaned)

train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
        
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [21]:
vocabulary_size

3016

In [22]:
len(n_sequences)

25642

In [23]:
train_inputs = n_sequences[:, :-1]
train_targets = n_sequences[:, -1]
train_targets = to_categorical(train_targets, num_classes = vocabulary_size)
seq_len = train_inputs.shape[1]
print(train_inputs.shape)
print(train_targets[0])

(25642, 3)
[0. 0. 0. ... 0. 0. 0.]


In [24]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 3)              9048      
                                                                 
 lstm_2 (LSTM)               (None, 3, 50)             10800     
                                                                 
 lstm_3 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 3016)              153816    
                                                                 
Total params: 196,414
Trainable params: 196,414
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
model.fit(train_inputs, train_targets, epochs=500, verbose=1)

Epoch 1/500
802/802 [==============================] - 15s 14ms/step - loss: 6.4646 - accuracy: 0.0514
Epoch 2/500
802/802 [==============================] - 9s 12ms/step - loss: 6.1824 - accuracy: 0.0519
Epoch 3/500
802/802 [==============================] - 9s 11ms/step - loss: 6.1479 - accuracy: 0.0519
Epoch 4/500
802/802 [==============================] - 10s 12ms/step - loss: 6.0704 - accuracy: 0.0519
Epoch 5/500
802/802 [==============================] - 11s 14ms/step - loss: 5.9382 - accuracy: 0.0530
Epoch 6/500
802/802 [==============================] - 11s 14ms/step - loss: 5.7490 - accuracy: 0.0627
Epoch 7/500
802/802 [==============================] - 10s 12ms/step - loss: 5.5926 - accuracy: 0.0744
Epoch 8/500
802/802 [==============================] - 10s 13ms/step - loss: 5.4747 - accuracy: 0.0795
Epoch 9/500
802/802 [==============================] - 10s 12ms/step - loss: 5.3765 - accuracy: 0.0851
Epoch 10/500
802/802 [==============================] - 11s 13ms/step - los

In [26]:
model.save('lstfm_kafka1kepochs.h5') 
#model.load('lstfm_kafka1kepochs.h5')

In [27]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

bird
Next word suggestion: agree
Next word suggestion: rested
Next word suggestion: crawl


In [28]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

house
Next word suggestion: despite
Next word suggestion: rested
Next word suggestion: simply


In [29]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

I
Next word suggestion: will
Next word suggestion: had
Next word suggestion: was


In [30]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

She
Next word suggestion: might
Next word suggestion: was
Next word suggestion: would


In [31]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

dog
Next word suggestion: agree
Next word suggestion: rested
Next word suggestion: crawl


In [32]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

girl
Next word suggestion: leaving
Next word suggestion: simply
Next word suggestion: despite


In [33]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

car
Next word suggestion: agree
Next word suggestion: rested
Next word suggestion: crawl


In [34]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

house
Next word suggestion: despite
Next word suggestion: rested
Next word suggestion: simply


In [35]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

it
Next word suggestion: she
Next word suggestion: were
Next word suggestion: and


In [36]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

and
Next word suggestion: he
Next word suggestion: would
Next word suggestion: was


In [37]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

for i in (model.predict(pad_encoded)[0]).argsort()[-5:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:", pred_word)

time
Next word suggestion: she
Next word suggestion: of
Next word suggestion: his
Next word suggestion: to
Next word suggestion: and
